In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from monodepthloss import MonodepthLoss
from depthencoder import depthencoder 
from depth_decoder import *
DEVICE = torch.device("cuda:0")


In [2]:
x = torch.randn(4, 3,640,256).to(DEVICE)
encoderdecoder = DepthDecoder().to(DEVICE)
xout = encoderdecoder(x.view(-1,3,640,256))
for key in xout.keys():
    print(f"{key} is {xout[key].size()}")


('disp', 3) is torch.Size([4, 1, 80, 32])
('disp', 2) is torch.Size([4, 1, 160, 64])
('disp', 1) is torch.Size([4, 1, 320, 128])
('disp', 0) is torch.Size([4, 1, 640, 256])


In [3]:
import numpy as np
import matplotlib.pyplot as plt
import torch.optim as optim
from tqdm import tqdm
import random
import movieparser
import time
from torchvision.utils import save_image
import torchvision

optimizer = optim.Adam(encoderdecoder.parameters(),lr=0.001)

# network.load_state_dict(torch.load('model2.t'))

training_data = np.load("numpy_img/skyscraper_lowres-1.npy", allow_pickle=True)

In [ ]:
testing = False
results = []
j = 0
mean = []
f= open(f"logs/results-{int(time.time())}.txt","w+")
epochs = 1
#loss_function = MonodepthLoss(n=4, SSIM_w=0.85,\
#                disp_gradient_w=0.1, lr_w=1).to(DEVICE)
loss_function = MonodepthLoss(n=4, SSIM_w=0.85,\
                disp_gradient_w=0.1, lr_w=1).to(DEVICE)
for epoch in range(epochs):
    samps = list(range(len(training_data)))
    random.shuffle(samps)
    for frame in tqdm(samps):
        LinNimg = torch.from_numpy(training_data[4999:5000,0]).type(torch.cuda.FloatTensor)
        RoutNimg = torch.from_numpy(training_data[4999:5000,1]).type(torch.cuda.FloatTensor)
        # LinNm, LinN, LinNp, RoutNm, RoutN, RoutNp = movieparser.getframeset(frameset, images)
        LinN = torch.div(LinNimg, 255).permute(0,3,1,2)
        RoutN = torch.div(RoutNimg, 255).permute(0,3,1,2)

        
        encoderdecoder.zero_grad()

        output1 = encoderdecoder(LinN.view(-1,3,256,640))
        output2 = encoderdecoder(RoutN.view(-1,3,256,640))
        '''output1 = encoderdecoder(torch.cat([LinNm.type(torch.cuda.FloatTensor), \
                         LinN.type(torch.cuda.FloatTensor), \
                         LinNp.type(torch.cuda.FloatTensor)], 1))
        output2 = encoderdecoder(torch.cat([RoutNm.type(torch.cuda.FloatTensor), \
                         RoutN.type(torch.cuda.FloatTensor), \
                         RoutNp.type(torch.cuda.FloatTensor)], 1))'''
        if(testing):
            plt.imshow(RoutNimg.cpu())
            plt.show()
            plt.imshow(output1[('disp', 0)][0].view(256, 640).cpu().detach().numpy(), 'gray')
            plt.show()
            break
        output = []
        for i in output1.keys():
            output.insert(0, torch.cat((output1[i], output2[i]), 1))
        loss = loss_function(output,[LinN.view(-1,3,256,640),RoutN.view(-1,3,256,640)])
        #criterion = nn.MSELoss()
        #loss = criterion(output[-1], torch.cat([LinN[:,0,:,:], \
        #                                        RoutN[:,0,:,:]], 0).view(-1,2,256,640))
        
        loss.backward()
        mean.append(loss.item())
        if j % 100 == 0:
            f.write(f"{round(sum(mean)/len(mean),5)}\n")
            f.flush()
            results.append(sum(mean)/len(mean))
            mean = []
        j += 1
        if False:#j % 500000 == 0:
            thetime = int(time.time())
            torch.save(encoderdecoder.state_dict(), f"encoderdecoder-{thetime}")
            save_image(RoutN.cpu(), \
                       f'imageout/{thetime}-left.png')
            save_image(output1[('disp', 0)][0].cpu(), \
                       f'imageout/{thetime}-depth.png')
        optimizer.step()
f.close()

 70%|██████▉   | 6970/10000 [12:54<05:23,  9.38it/s]

In [ ]:
plt.imshow(output1[('disp', 0)][0].view(256, 640).cpu().detach().numpy(), 'gray')
plt.show()
plt.imshow(RoutN.cpu())

In [ ]:
LinN = torch.from_numpy(training_data[3000][0]).type(torch.cuda.FloatTensor)

plt.imshow(torch.div(LinN.cpu(),255))
plt.show()

In [ ]:
LinNimg = torch.from_numpy(training_data[1:5,0]).type(torch.cuda.FloatTensor)
print(f"before: {LinNimg.size()}")
print(f"after: {LinNimg.permute(0,3,2,1).size()}")


In [ ]:
LinNimg = torch.from_numpy(training_data[4999:5000,1]).type(torch.cuda.FloatTensor)
LinN = torch.add(torch.div(LinNimg, 255),-.5).permute(0,3,1,2)
plt.imshow(LinN.view(3,256,640).permute(1,2,0).cpu())
plt.show()